In [58]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from typing import List
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from pydantic import BaseModel, Field
import json
from langchain_core.agents import AgentActionMessageLog,AgentFinish
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

In [3]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [4]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-4")

In [9]:
# Load Documents

loader=TextLoader(file_path="data/state_of_the_union.txt",encoding="utf-8")
documents=loader.load()

In [11]:
# Split Documents into Chunks
textSplitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts=textSplitter.split_documents(documents=documents)

In [18]:
# Illustration
texts[4]

Document(metadata={'source': 'data/state_of_the_union.txt', 'page_chunk': 4}, page_content='And tonight I am announcing that we will join our allies in closing off American air space to all Russian flights – further isolating Russia – and adding an additional squeeze –on their economy. The Ruble has lost 30% of its value. \n\nThe Russian stock market has lost 40% of its value and trading remains suspended. Russia’s economy is reeling and Putin alone is to blame. \n\nTogether with our allies we are providing support to the Ukrainians in their fight for freedom. Military assistance. Economic assistance. Humanitarian assistance. \n\nWe are giving more than $1 Billion in direct assistance to Ukraine. \n\nAnd we will continue to aid the Ukrainian people as they defend their country and to help ease their suffering.  \n\nLet me be clear, our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  \n\nOur forces are not going to Europe to fight in Ukraine, but 

In [19]:
# Illustration
texts[4].metadata["page_chunk"]=4

In [20]:
# Illustration
texts[4]

Document(metadata={'source': 'data/state_of_the_union.txt', 'page_chunk': 4}, page_content='And tonight I am announcing that we will join our allies in closing off American air space to all Russian flights – further isolating Russia – and adding an additional squeeze –on their economy. The Ruble has lost 30% of its value. \n\nThe Russian stock market has lost 40% of its value and trading remains suspended. Russia’s economy is reeling and Putin alone is to blame. \n\nTogether with our allies we are providing support to the Ukrainians in their fight for freedom. Military assistance. Economic assistance. Humanitarian assistance. \n\nWe are giving more than $1 Billion in direct assistance to Ukraine. \n\nAnd we will continue to aid the Ukrainian people as they defend their country and to help ease their suffering.  \n\nLet me be clear, our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  \n\nOur forces are not going to Europe to fight in Ukraine, but 

In [21]:
# Add in the fake source information (how is covered in the illustrations above)
for i, doc in enumerate(texts):
    doc.metadata["page_chunk"]=i

In [24]:
# Create Vectorstore and retriever
embeddings=SentenceTransformerEmbeddings()
vectorstore=FAISS.from_documents(documents=texts,embedding=embeddings)
retriever=vectorstore.as_retriever()

C:\Users\MSI 1\AppData\Local\Temp\ipykernel_8696\2889723379.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=SentenceTransformerEmbeddings()


In [26]:
# Create Retriever Tool

retrieverTool=create_retriever_tool(
    retriever=retriever,
    name="state-of-union-retriever",
    description="Query a retriever to get information about state of the union address"
)

In [28]:
class Response(BaseModel):
    """
        Final response to the question being asked
    """
    answer: str=Field(
        description="The final answer to respond to the user"
    )

    sources: List[int]=Field(
        description="""
            List of page chunks that contain answer to the question.
            Only include a page chunk if it contains a relevant information
        """
    )

In [49]:
llmWithTools=llmOpenAI.bind_functions(functions=[retrieverTool,Response])

In [35]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a helpful assistant"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [59]:
def parse(output):
    # If no function was involved, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(
            return_values={"output":output.content},
            log=output.content
        )
    # Else Parse out the content
    else:
        function_call=output.additional_kwargs['function_call']
        name=function_call['name']
        inputs=json.loads(function_call['arguments'])

        # If the Response Function was invoked, return to the user with the function inputs
        if name=="Response":
            return AgentFinish(return_values=inputs,log=str(function_call))
        # if any other function is called, treat it as a tool invocation and return an agent action
        else:
            return AgentActionMessageLog(tool=name, tool_input=inputs, log="",message_log=[output])
            

In [60]:
# Creating the Custom Agent
agent={
    "input":lambda x:x['input'],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(x['intermediate_steps'])
    } | prompt | llmWithTools | parse

In [61]:
agentExecutor=AgentExecutor(agent=agent, tools=[retrieverTool],verbose=True)

In [62]:
agentExecutor.invoke(
    input={"input":"What did the President say about Putin?"},
)



> Entering new AgentExecutor chain...
Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.

And I’m taking robu

{'input': 'What did the President say about Putin?',
 'answer': "The President spoke about Putin's invasion of Ukraine, stating that Putin miscalculated his actions and instead of the world bending to his will, he met a wall of strength. He mentioned that robust actions are being taken to target Russia's economy with sanctions and that the United States has worked with other countries to release oil from reserves around the world. The President also mentioned that the U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs. He emphasized that Putin is now more isolated from the world than ever and his war on Ukraine will leave Russia weaker and the rest of the world stronger. He ended by stating that Putin may circle Kyiv with tanks, but he will never gain the hearts and souls of the Ukrainian people.",
 'sources': [0]}

In [63]:
agentExecutor.invoke(
    input={"input":"What did the President say about Ketanji brown Jackson?"},
)



> Entering new AgentExecutor chain...
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since 

{'input': 'What did the President say about Ketanji brown Jackson?',
 'answer': "President honored Justice Stephen Breyer and then spoke about his recent nomination of Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court. He described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He highlighted her background as a former top litigator in private practice, a former federal public defender and coming from a family of public school educators and police officers. He also mentioned that since her nomination, she has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.",
 'sources': [0]}

In [73]:
for content in agentExecutor.iter(
    inputs={"input":"What did the President say about Putin?"},
    
):
    print(content,"-\n------")



> Entering new None chain...
Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.

And I’m taking robust action